In [17]:
# input
n_topics = 10

In [18]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

# OK, let's finally do some topic modelling

In [19]:
TM_MODELS_PATH = '/media/pablo/data/Tesis/models/old/tm_feats/'
from os.path import join

In [20]:
dictionary = gensim.corpora.Dictionary.load(join(TM_MODELS_PATH,"tweets_es.dict"))

with open(join(TM_MODELS_PATH,'./tweets_es_bow.pickle'), 'rb') as f:
    bow = pickle.load(f)

In [21]:
len(dictionary.dfs)

11238

In [22]:
from math import ceil
iters = 100
passes = 10
workers = 7
chunksize = int(ceil(len(bow)*1.0/workers))

In [23]:
model_path = join(TM_MODELS_PATH,"./tweets_es_%dtopics.lda" % n_topics) 

In [8]:
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=chunksize,
        workers=workers
)
model.save(model_path)

In [24]:
model = gensim.models.LdaModel.load(model_path)

In [ ]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("../tweets_es_%d.viz" % n_topics,'wb'))

In [ ]:
viz = pickle.load(open("../tweets_es_%d.viz" % n_topics,'rb'))

In [ ]:
pyLDAvis.display(viz)

# Términos relevantes por tema

In [ ]:
feature_names = [dictionary[i] for i in range(len(dictionary))]

In [ ]:
model.show_topics(n_topics)

# Extracción de features LDA sobre tweets

In [25]:
tweets_lda = [model[t_bow] for t_bow in bow]

In [26]:
def rows_to_csc(rows):
    data = []
    row_ind = []
    col_ind = []
    for i, r in enumerate(rows):
        for j, d in r:
            row_ind.append(i)
            col_ind.append(j)
            data.append(d)
    return csc_matrix((data, (row_ind, col_ind)))

In [27]:
from scipy.sparse import csc, csc_matrix

In [28]:
X_tweets_lda = rows_to_csc(tweets_lda)

In [29]:
s = open_session()
all_tweets_es_ids = [t[0] for t in s.query(Tweet.id, Tweet.lang).all() if t.lang == 'es']
len(all_tweets_es_ids)    

1636480

In [30]:
X_tweets_lda_df = pd.DataFrame(index=all_tweets_es_ids, data=X_tweets_lda.todense())

In [31]:
X_tweets_lda_df.to_pickle(join(TM_MODELS_PATH, "./alltweets_es_lda%d.pickle" % n_topics))